In [2]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [574]:
import pandas as pd
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup
import requests

### Extracting the links

In [575]:
url_pages = []
# urls_list = []
for i in range(0,116):
    url = f"https://www.contractsfinder.service.gov.uk/Search/Results?&page={i}#dashboard_notices"
    url_pages.append(url)
for m in url_pages:
    response = requests.get(m)
    page_contents = response.text
    doc = BeautifulSoup(page_contents,"html.parser")

In [263]:
url_pages

['https://www.contractsfinder.service.gov.uk/Search/Results?&page=0#dashboard_notices',
 'https://www.contractsfinder.service.gov.uk/Search/Results?&page=1#dashboard_notices',
 'https://www.contractsfinder.service.gov.uk/Search/Results?&page=2#dashboard_notices',
 'https://www.contractsfinder.service.gov.uk/Search/Results?&page=3#dashboard_notices',
 'https://www.contractsfinder.service.gov.uk/Search/Results?&page=4#dashboard_notices',
 'https://www.contractsfinder.service.gov.uk/Search/Results?&page=5#dashboard_notices',
 'https://www.contractsfinder.service.gov.uk/Search/Results?&page=6#dashboard_notices',
 'https://www.contractsfinder.service.gov.uk/Search/Results?&page=7#dashboard_notices',
 'https://www.contractsfinder.service.gov.uk/Search/Results?&page=8#dashboard_notices',
 'https://www.contractsfinder.service.gov.uk/Search/Results?&page=9#dashboard_notices',
 'https://www.contractsfinder.service.gov.uk/Search/Results?&page=10#dashboard_notices',
 'https://www.contractsfinder.s

In [576]:
datas = []
for i in url_pages:
    response = requests.get(i)
    page_contents = response.content
    doc = BeautifulSoup(page_contents,"html.parser")
    data = doc.find_all("div",{"class":"search-result-entry"})
    for d in data:
        datas.append(list(d.stripped_strings))


In [577]:
lists = []
for d in range(len(datas)):
#     for i in datas
    lists.append(datas[d][0])

In [578]:
lists = set(lists)
key = list(lists)
key.sort()
keys = []
for i in key:
    print(i)
    keys.append(i)

Approach to market date
Closing
Contract location
Contract value
Notice status
Procurement stage
Publication date


In [579]:
main_content=[]
for i in url_pages:
    response = requests.get(url)
    page_contents = response.content
    doc = BeautifulSoup(page_contents,"html.parser")
    content = doc.find_all(class_="search-result")
    for i in headings:
        main_content.append(list(i.stripped_strings))

In [580]:
Heading = []
Sub_heading = []
for m in main_content:
    Heading.append(m[0])
    Sub_heading.append(m[1])

In [581]:
Approach_to_market_date = []
Closing = []
Contract_location = []
Contract_value = []
Notice_status = []
Procurement_stage = []
Publication_date = []
for m in main_content:
    if keys[0]in m:
        iloc = m.index(keys[0])+1
        Approach_to_market_date.append(m[iloc])
    else:
        Approach_to_market_date.append(np.nan)
    if keys[1]in m:
        iloc = m.index(keys[1])+1
        Closing.append(m[iloc])
    else:
        Closing.append(np.nan)
    if keys[2]in m:
        iloc = m.index(keys[2])+1
        Contract_location.append(m[iloc])
    else:
        Contract_location.append(np.nan)
    if keys[3]in m:
        iloc = m.index(keys[3])+1
        Contract_value.append(m[iloc])
    else:
        Contract_value.append(np.nan)
    if keys[4]in m:
        iloc = m.index(keys[4])+1
        Notice_status.append(m[iloc])
    else:
        Notice_status.append(np.nan)
    if keys[5]in m:
        iloc = m.index(keys[5])+1
        Procurement_stage.append(m[iloc])
    else:
        Procurement_stage.append(np.nan)
    if keys[6]in m:
        iloc = m.index(keys[6])+1
        Publication_date.append(m[iloc])
    else:
        Publication_date.append(np.nan)

### Creating a DataFrame

In [582]:
df = pd.DataFrame({"Heading":Heading,"Sub_Heading":Sub_heading,"Closing":Closing,"Notice_status":Notice_status,"Contract_location":Contract_location,"Contract_value":Contract_value,"Publication_date":Publication_date,"Procurement_stage":Procurement_stage,"Approach_to_market_date":Approach_to_market_date})

In [583]:
df.head()

,Heading,Sub_Heading,Closing,Notice_status,Contract_location,Contract_value,Publication_date,Procurement_stage,Approach_to_market_date
0,Future Architecture for Submarine Technology t...,Ministry of Defence,8 July 2022,Open,PO2 8BY,NaN,2 July 2022,Early engagement,NaN
1,Provision of Cleaning Services to Residential ...,University of Leeds,"4 August 2022, 12am",Open,Yorkshire and the Humber,"£4,500,000",2 July 2022,Opportunity,NaN
2,NHS Framework Agreement for the supply of Adul...,NHS ENGLAND,NaN,Open,"England, Isle of Man, Channel Islands","£160,000,000","17 March 2022, last edited 2 July 2022",Future opportunity,1 April 2023
3,NHS Framework Agreement for the supply of Adul...,NHS ENGLAND,NaN,Open,"England, Isle of Man, Channel Islands","£160,000,000","17 March 2022, last edited 2 July 2022",Future opportunity,1 April 2024
4,NHS Framework Agreement for the supply of Adul...,NHS ENGLAND,NaN,Open,"England, Isle of Man, Channel Islands","£160,000,000","14 September 2021, last edited 2 July 2022",Future opportunity,1 August 2022


In [584]:
df["Approach_to_market_date"] = pd.to_datetime(df["Approach_to_market_date"])

In [585]:
df["Closing"] = pd.to_datetime(df["Closing"])

In [586]:
vals = []
for i in df["Contract_value"]:
    if type(i)!=float:
        x = i.replace("£","")
        z = x.replace(",","")
        y = z.split("to")
        num_list = []
        for num in y:
            num_list.append((int(num)))
        vals.append((sum(num_list)/len(num_list))*95.75)
    else:
        vals.append(i*95.57)
df["Contract_value"] = vals

In [587]:
df.to_csv("Internship_task.csv")

### Geocoding

In [568]:
!pip install geopandas
!pip install geopy


    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\hp\anaconda3\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\hp\\AppData\\Local\\Temp\\pip-install-rz4h8shn\\fiona_b2e7fafaa41246eca5693b196aa31264\\setup.py'"'"'; __file__='"'"'C:\\Users\\hp\\AppData\\Local\\Temp\\pip-install-rz4h8shn\\fiona_b2e7fafaa41246eca5693b196aa31264\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\hp\AppData\Local\Temp\pip-pip-egg-info-rcrr2nni'
         cwd: C:\Users\hp\AppData\Local\Temp\pip-install-rz4h8shn\fiona_b2e7fafaa41246eca5693b196aa31264\
    Complete output (1 lines):
    A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment variable or use


  Running setup.py clean for fiona
Failed to build fiona
    Running setup.py install for fiona: started
    Running setup.py install for fiona: finished with status 'error'


In [573]:
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
locator = Nominatim(user_agent="myGeocoder")
# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df['Geocodes'] = df['Contract_location'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df['point'] = df['Geocodes'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

KeyboardInterrupt: 